In [1]:
from openfisca_core import columns
from openfisca_core import periods
from openfisca_core import reforms
import openfisca_france
from openfisca_france.model.base import *
from numpy import maximum as max_, logical_not as not_, logical_or as or_
from datetime import date

In [2]:
tax_benefit_system = openfisca_france.init_country()()

In [3]:
Reform_individualisation = reforms.make_reform(
    key = 'individualisation',
    name = u'Individualisation du RSA et de l\'IRPP',
    reference = tax_benefit_system,
    )

In [4]:
def modify_legislation_json(reference_legislation_json_copy):
    #reference_legislation_json_copy['children']['xxx']['values'][0]['value'] = 0
    return reference_legislation_json_copy

In [5]:
periods.period(2014)

Period((u'year', Instant((2014, 1, 1)), 1))

In [6]:
class nb_enfant_rsa_indiv(Reform_individualisation.Variable):
    entity_class = Individus
    column = columns.FloatCol
    label = u"Nombre d'enfants de la famille"
    def function(self, simulation, period):
        #variable = Familles.column_by_name['nb_enfant_rsa']
        period = period.this_month
        
        return period, simulation.calculate("nb_enfant_rsa", period)

In [7]:
reform = Reform_individualisation()
reform.modify_legislation_json(modifier_function = modify_legislation_json)
scenario = reform.new_scenario()
scenario.init_single_entity(
    period = 2012,
    parent1 = dict(
        age = 30,
        salaire_de_base = 15000,
        ),
    enfants = [
        dict(age = 10),
        dict(age = 12),
        dict(age = 18),
        ],
    )



In [8]:
simulation = scenario.new_simulation()

In [16]:
print simulation.calculate("nb_enfant_rsa", "2016-01").shape
print simulation.calculate("salaire_net", "2016-01").shape

(1,)
(4,)


In [9]:
simulation.calculate("nb_enfant_rsa_indiv", "2012")

AssertionError: Function nb_enfant_rsa_indiv@individus<2012>() --> <2012-01>[3] returns an array of size 1, but size 4 is expected for individu

In [ ]:
%debug

> /home/openfisca/openfisca-core/openfisca_core/holders.py(137)compute()
    136                     "Requested period {} differs from {} returned by variable {}".format(period,
--> 137                         formula_dated_holder.period, column.name)
    138             return formula_dated_holder

ipdb> period
Period((u'year', Instant((2012, 1, 1)), 1))
ipdb>  formula_dated_holder.period
Period((u'month', Instant((2012, 1, 1)), 1))
ipdb> u
> /home/openfisca/openfisca-core/openfisca_core/simulations.py(157)compute()
    156         holder = self.get_or_new_holder(column_name)
--> 157         return holder.compute(period = period, **parameters)
    158 

ipdb> period
Period((u'year', Instant((2012, 1, 1)), 1))
ipdb> u
> /home/openfisca/openfisca-france/openfisca_france/model/prestations/minima_sociaux/rsa.py(494)function()
    493 
--> 494         return period, self.sum_by_entity(simulation.compute('rsa_enfant_a_charge', period))
    495 

ipdb> period
Period((u'year', Instant((2012, 

In [13]:
import numpy as np
np.array([1,2,4])- np.array([1,2,4,7])

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

In [6]:
class rsa_eligibilite_tns_indiv(Reform_individualisation.Variable):
    # ATTENTION, on a fait le choix de se caler sur les célibataires dans le calcul du plafond,
    # ce qui a pour effet de rendre inéligibles certains couples ou certaines familles avec enfants
    column = BoolCol
    entity_class = Individus
    label = u"Eligibilité au RSA pour un travailleur non salarié"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')

        #tns_benefice_exploitant_agricole_holder = simulation.compute('tns_benefice_exploitant_agricole', period)
        #tns_benefice_exploitant_agricole = self.sum_by_entity(tns_benefice_exploitant_agricole_holder)
        tns_benefice_exploitant_agricole = simulation.calculate('tns_benefice_exploitant_agricole', period) #new
        #tns_employe_holder = simulation.compute('tns_employe', period)
        #tns_employe = self.any_by_roles(tns_employe_holder)
        tns_employe = simulation.calculate('tns_employe', period) #new
        #tns_autres_revenus_chiffre_affaires_holder = simulation.compute('tns_autres_revenus_chiffre_affaires', period)
        #tns_autres_revenus_chiffre_affaires = self.split_by_roles(tns_autres_revenus_chiffre_affaires_holder)
        tns_autres_revenus_chiffre_affaires = simulation.calculate('tns_autres_revenus_chiffre_affaires', period) #new
        #tns_autres_revenus_type_activite_holder = simulation.compute('tns_autres_revenus_type_activite', period)
        #tns_autres_revenus_type_activite = self.split_by_roles(tns_autres_revenus_type_activite_holder)
        tns_autres_revenus_type_activite = simulation.calculate('tns_autres_revenus_type_activite', period) #new
        
        #has_conjoint = simulation.calculate('nb_par', period) > 1
        #nb_enfant_rsa = simulation.calculate('nb_enfant_rsa', period)
        P = simulation.legislation_at(period.start)
        P_agr = P.tns.exploitant_agricole
        P_micro = P.ir.rpns.microentreprise
        maj_2p = P_agr.maj_2p
        maj_1e_2ad = P_agr.maj_1e_2ad
        maj_e_sup = P_agr.maj_e_sup

        #def eligibilite_agricole(has_conjoint, nb_enfant_rsa, tns_benefice_exploitant_agricole, P_agr):
        def eligibilite_agricole(tns_benefice_exploitant_agricole, P_agr):
            plafond_benefice_agricole = P_agr.plafond_rsa * P.cotsoc.gen.smic_h_b
            #taux_avec_conjoint = 1 + maj_2p + maj_1e_2ad * (nb_enfant_rsa > 0) + maj_e_sup * max_(nb_enfant_rsa - 1, 0)
            #taux_sans_conjoint = 1 + maj_2p * (nb_enfant_rsa > 0) + maj_e_sup * max_(nb_enfant_rsa - 1, 0)
            #taux_majoration = has_conjoint * taux_avec_conjoint + (1 - has_conjoint) * taux_sans_conjoint
            taux_majoration = 1 #new
            plafond_benefice_agricole_majore = taux_majoration * plafond_benefice_agricole

            return tns_benefice_exploitant_agricole < plafond_benefice_agricole_majore

        def eligibilite_chiffre_affaire(ca, type_activite, P_micro):
            plaf_vente = P_micro.vente.max
            plaf_service = P_micro.servi.max

            return ((type_activite == 0) * (ca <= plaf_vente)) + ((type_activite >= 1) * (ca <= plaf_service))

        eligibilite_agricole = eligibilite_agricole(
       #     has_conjoint, nb_enfant_rsa, tns_benefice_exploitant_agricole, P_agr
            tns_benefice_exploitant_agricole, P_agr
            )
        eligibilite_chiffre_affaire = (
            eligibilite_chiffre_affaire(
       #         tns_autres_revenus_chiffre_affaires[CHEF], tns_autres_revenus_type_activite[CHEF], P_micro
                tns_autres_revenus_chiffre_affaires, tns_autres_revenus_type_activite, P_micro #new
            ) *
            eligibilite_chiffre_affaire(
       #         tns_autres_revenus_chiffre_affaires[PART], tns_autres_revenus_type_activite[PART], P_micro
                tns_autres_revenus_chiffre_affaires, tns_autres_revenus_type_activite, P_micro #new
            )
        )

        return period, eligibilite_agricole * (1 - tns_employe) * eligibilite_chiffre_affaire


In [7]:
class rsa_eligibilite_indiv(Reform_individualisation.Variable):
    # On maintient un calcul en fonction du ménage : règle du RSA pour les moins de 25 ans maintenu (s'ils ont des enfants)
    column = BoolCol
    entity_class = Individus
    label = u"Eligibilité au RSA"

    def function(self, simulation, period):
        period = period.this_month
        #age_holder = simulation.compute('age', period)
        #age_parents = self.split_by_roles(age_holder, roles = [CHEF, PART])
        age = simulation.calculate('age', period) #new
        #activite_holder = simulation.compute('activite', period)
        #activite_parents = self.split_by_roles(activite_holder, roles = [CHEF, PART])
        activite = simulation.calculate('activite', period) #new
        #nb_enfant_rsa = simulation.calculate('nb_enfant_rsa', period)
        nb_enfant_rsa_indiv = simulation.calculate('nb_enfant_rsa_indiv', period)
        rsa_eligibilite_tns_indiv = simulation.calculate('rsa_eligibilite_tns_indiv', period)
        #rsa_condition_nationalite = simulation.compute('rsa_condition_nationalite', period)
        #condition_nationalite = self.any_by_roles(rsa_condition_nationalite, roles = [CHEF, PART])
        condition_nationalite = simulation.calculate('rsa_condition_nationalite', period) #new
        rmi = simulation.legislation_at(period.start).minim.rmi
        #age_min = (nb_enfant_rsa_indiv == 0) * rmi.age_pac
        quifam = simulation.calculate('quifam', period)
        condition_age = or_(age >= rmi.age_pac,  (age < rmi.age_pac) * (quifam == 0) * (nb_enfant_rsa_indiv > 1))
        # ce qui est après le or_ a été codé pour le RSA jeune, mais je ne suis pas sûr du tout que ce soit la vraie règle

        #eligib = (
        #    (age_parents[CHEF] >= age_min) * not_(activite_parents[CHEF] == 2) +
        #    (age_parents[PART] >= age_min) * not_(activite_parents[PART] == 2)
        #)
        
        eligib = condition_age * not_(activite == 2) * (
            condition_nationalite *
            rsa_eligibilite_tns_indiv
            )

        return period, eligib

In [8]:
class nb_par_indiv(Reform_individualisation):
    entity_class = Individus
    label = u"Nombre de parents de la famille"
    variable = Familles.column_by_name['nb_par']

In [9]:
#class af_nbenf_indiv(Reform_individualisation):
#    entity_class = Individus
#    label = u"Nombre d'enfants de la famille (au sens des allocations familiales)"
#    variable = Familles.column_by_name['af_nbenf']

In [10]:
class supp_monoparental(Reform_individualisation):
    label = u"Supplément RSA monoparental"
    column = FloatCol
    entity_class = Individus
    
    def function(self, simulation, period):
        quifam = simulation.calculate('quifam', period)
        nb_enfant_rsa_indiv = simulation.calculate('nb_enfant_rsa_indiv', period)
        nb_par_indiv = simulation.calculate('nb_par_indiv', period)
        rmi = simulation.legislation_at(period.start).minim.rmi
        af = simulation.legislation_at(period.start).fam.af
        
        return period, self.zeros() + (quifam == 0) * (nb_par_indiv == 1) * (nb_enfant_rsa_indiv > 0) * (rmi.rmi * rmi.txp2 - af.bmaf * 0.41)

In [11]:
class aspa_asi_ass_famille(Reform_individualisation):
    # Comme on ne pouvait pas individualiser ces trois prestations, on a récupéré les prestations familialisées pour le calcul du RSA individuel
    entity_class = Individus
    label = u"Somme de l'ASPA, ASI et ASS de la famille"
    variable = Familles.column_by_name['br_rmi_ms']

KeyError: 'br_rmi_ms'

In [12]:
class rsa_indiv(Reform_individualisation.Variable):
    # Concernant les minimas sociaux qui sont déduits du RSA : - minimas sociaux individuels (aah, caah) : pas de problème
    #   - minimas sociaux familialisés (aspa, asi, ass) : on a pris en compte l'allocation familialisée ; il faudrait idéalement les individualiser (mais c'est une autre réforme)
    label = u"RSA individualisé"
    column = FloatCol
    entity_class = Individus
    
    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        eligib = simulation.calculate('rsa_eligibilite_indiv', period)
        ra_rsa_i = simulation.calculate('ra_rsa_i', period)
        br_rmi_i = simulation.calculate('br_rmi_i', period)
        aah = simulation.calculate('aah', period)
        caah = simulation.calculate('caah', period)        
        aspa_asi_ass_famille = simulation.calculate('aspa_asi_ass_famille', period) 
        supp_monoparental = simulation.calculate('supp_monoparental', period)
        rmi = simulation.legislation_at(period.start).minim.rmi
        br_rmi_indiv = br_rmi_i + ra_rsa_i + aspa_asi_ass_famille + aah + caah
        
        return period, self.zeros() + eligib * (supp_monoparental + max_(rmi.rmi - rmi.rmi * rmi.forfait_logement.taux1 - br_rmi_indiv + rmi.pente * ra_rsa_i, 0) )

In [13]:
reform = Reform_individualisation()
reform.modify_legislation_json(modifier_function = modify_legislation_json)

In [14]:
scenario = reform.new_scenario()

In [15]:
scenario.init_single_entity(
    period = 2012,
    parent1 = dict(
        age = 30,
        salaire_de_base = 15000,
        ),
    enfants = [
        dict(age = 10),
        dict(age = 12),
        dict(age = 18),
        ],
    )

In [16]:
reform_simulation = scenario.new_simulation()

In [17]:
reform_simulation.calculate('rsa', "2015-01")

array([ 817.75152588], dtype=float32)

In [18]:
reform_simulation.calculate_add('rsa_indiv', "2015-01")

ERROR:openfisca_core.formulas:An error occurred while calling formula rsa_eligibilite_indiv@individus<2015-01> in module __main__
ERROR:openfisca_core.formulas:An error occurred while calling formula rsa_indiv@individus<2015-01> in module __main__


KeyError: 'nb_enfant_rsa_indiv'

In [ ]:
scenario.init_single_entity(
    period = 2015,
    parent1 = dict(
        age = 30,
        salaire_de_base = 0,
        ),
    enfants = [
        dict(birth = date(2005,1,1)),
        dict(birth = date(2005,1,1)),
        ],
    )

In [ ]:
reform_simulation2 = scenario.new_simulation()

In [ ]:
reform_simulation2.calculate('rsa', "2015-12")

In [ ]:
reform_simulation2.calculate('rsa_indiv', "2015-12")

In [ ]:
reform_simulation2.calculate('age', "2015-03")

In [ ]:
reform_simulation.calculate('rsa_indiv', "2012-12")

# Graphiques

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from datetime import date  # module nécessaire pour la définition des dates, dont notamment les dates de naissances
from openfisca_france.tests.base import tax_benefit_system




def make_couple_with_child_scenario(nombre_enfants = 0, year = None, tax_benefit_system = tax_benefit_system,
                                    axes_variable = 'salaire_de_base',  ax_variable_max = 150000, count = 5000):
    enfant = [dict(
        birth = date(2005, 1, 1),
        )]
    enfants = enfant * nombre_enfants
    scenario = tax_benefit_system.new_scenario().init_single_entity(
        axes = [[
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-2,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-1,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year,
                ),
            ]],
        period = year,
        parent1 = dict(
            birth = date(1980, 1, 1),
            statmarit = 5, #pacsés
            ),
        parent2 = dict(
            birth = date(1980, 1, 1),
            statmarit = 5,
            ),
        enfants = enfants,
        menage = dict(
            loyer = 1000,
            statut_occupation = 4,
            ),
        )
    return scenario
def make_single_with_child_scenario(nombre_enfants = 1, year = None, tax_benefit_system = tax_benefit_system,
                                    axes_variable = 'salaire_de_base',  ax_variable_max = 150000, count = 5000):
    enfant = [dict(
        birth = date(2005, 1, 1),
        )]
    enfants = enfant * nombre_enfants
    scenario = tax_benefit_system.new_scenario().init_single_entity(
        axes = [[
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-2,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-1,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year,
                ),
            ]],
        period = year,
        parent1 = dict(
            birth = date(1980, 1, 1),
            ),
        enfants = enfants,
        menage = dict(
            loyer = 1000,
            statut_occupation = 4,
            ),
        )
    return scenario

In [ ]:
scenario_monoparental_actuel = make_single_with_child_scenario(2, 2014)

In [ ]:
scenario_monoparental_reform = make_single_with_child_scenario(2, 2014, reform)

In [ ]:
simulation_monoparental_actuel = scenario_monoparental_actuel.new_simulation()

In [ ]:
simulation_monoparental_reform = scenario_monoparental_reform.new_simulation()

In [ ]:
salaire_net_actuel = simulation_monoparental_actuel.calculate_add("salaire_net")

In [ ]:
salaire_net_reform = simulation_monoparental_reform.calculate_add("salaire_net")

In [ ]:
rsa_actuel = simulation_monoparental_actuel.calculate_add("rsa")

In [ ]:
rsa_reform = simulation_monoparental_reform.calculate_add("rsa_indiv")

In [ ]:
revdisp_actuel = simulation_monoparental_actuel.calculate_add("revdisp")

In [ ]:
revdisp_reform = simulation_monoparental_reform.calculate_add("revdisp")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(salaire_net_actuel[::3],
        rsa_actuel, salaire_net_reform[::3],
        rsa_reform[::3])

# Revenu de base enfant

In [ ]:
from openfisca_core import reforms
ReformeRevenuDeBaseEnfants = reforms.make_reform(
    key = 'revenu_de_base_enfants',
    name = u"Réforme Revenu de base enfants",
    reference = tax_benefit_system,
    )

# Création d'un revenu de base enfant - Version famille

from openfisca_france.model.prestations.prestations_familiales.base_ressource import nb_enf

class rdb_enfant_famille(Reform_individualisation):
    column = FloatCol
    entity_class = Familles
    label = u"Revenu de base enfant"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age_holder = simulation.compute('age', period)
        P = simulation.legislation_at(period.start).fam.af        
        bmaf = P.bmaf

        
        smic55_holder = simulation.compute('smic55', period)
        smic55 = self.split_by_roles(smic55_holder, roles = ENFS)        
        age = self.split_by_roles(age_holder, roles = ENFS)
        
        smic5 = {
            role: array * 0
            for role, array in smic55.iteritems()
            }
        nbenf_inf13 = nb_enf(age, smic5, 0, 13)
        nbenf_sup14 = nb_enf(age, smic5, 14, 18)
        
        return period, (nbenf_inf13 * 0.41 + nbenf_sup14 * 0.57) * bmaf

# Les taux 0,41 et 0,16 (0,57-0,41) sont issus des allocations familiales


# In[13]:

# Création d'un revenu de base enfant - Version individus

class rdb_enf(Reform_individualisation):
    column = FloatCol
    entity_class = Individus
    label = u"Revenu de base enfant"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age = simulation.calculate('age')
        P = simulation.legislation_at(period.start).fam.af        
        bmaf = P.bmaf
        
        return period, ((age < 14) * 0.41 + not_(age < 14) * 0.57) * bmaf * (age <= 18)


# Individualisation de l'impôt

In [ ]:

Reform = reforms.make_reform(
    key = 'plafonnement_gain_quotient_conjugal',
    name = u"plafonnement_gain_quotient_conjugal",
    reference = tax_benefit_system,
    )


# In[12]:

class rev_cat_tspr_individuel(Reform_individualisation):
    column = FloatCol
    entity_class = Individus
    label = u"Nouvelles cotisations contributives"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'year').period('year')
        tspr = simulation.calculate('tspr', period)
        indu_plaf_abat_pen = simulation.calculate('indu_plaf_abat_pen_individuel', period)
        return period, tspr + indu_plaf_abat_pen
    
class indu_plaf_abat_pen_individuel(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"indu_plaf_abat_pen"

    def function(self, simulation, period):
        """
        Plafonnement de l'abattement de 10% sur les pensions du foyer
        'foy'
        """
        period = period.start.offset('first-of', 'year').period('year')
        abatpen = simulation.legislation_at(period.start).ir.tspr.abatpen

        pen_net = simulation.calculate('pen_net', period)
        rev_pen = simulation.calculate('rev_pen', period)
        print type(pen_net)

        abat = rev_pen - pen_net
        return period, abat - min_(abat, abatpen.max)


# In[13]:

class rev_cat_individuel(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenus catégoriels"
    url = "http://www.insee.fr/fr/methodes/default.asp?page=definitions/revenus-categoriesl.htm"

    def function(self, simulation, period):
        '''
        Revenus Categoriels
        '''
        period = period.start.offset('first-of', 'year').period('year')
        rev_cat_tspr_individuel = simulation.calculate('rev_cat_tspr_individuel', period)
       # rev_cat_rvcm = simulation.calculate('rev_cat_rvcm', period)
       # rev_cat_rfon = simulation.calculate('rev_cat_rfon', period)
       # rev_cat_rpns = simulation.calculate('rev_cat_rpns', period)
       # rev_cat_pv = simulation.calculate('rev_cat_pv', period)

        return period, rev_cat_tspr_individuel # + rev_cat_rvcm + rev_cat_rfon + rev_cat_rpns + rev_cat_pv TODO :Add everything


# In[14]:

class rbg_individuel(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu brut global"
    url = "http://www.documentissime.fr/dossiers-droit-pratique/dossier-19-l-impot-sur-le-revenu-les-modalites-generales-d-imposition/la-determination-du-revenu-imposable/le-revenu-brut-global.html"

    def function(self, simulation, period):
        '''Revenu brut global
        '''
        period = period.start.offset('first-of', 'year').period('year')
        rev_cat_individuel = simulation.calculate('rev_cat_individuel', period)
       # deficit_ante = simulation.calculate('deficit_ante', period)
      #  f6gh = simulation.calculate('f6gh', period)
        #nbic_impm_individuel = simulation.calculate('nbic_impm', period)
        #nacc_pvce_individuel = simulation.calculate('nacc_pvce', period)
        cga = simulation.legislation_at(period.start).ir.rpns.cga_taux2

        # (Total 17)
        # sans les revenus au quotient
        #nacc_pvce = self.sum_by_entity(nacc_pvce_holder)
        return period, max_(0, rev_cat_individuel) # + f6gh + nbic_impm_individuel + nacc_pvce) * (1 + cga) - deficit_ante


# In[15]:

class rng_individuel(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu net global"
    url = "http://impotsurlerevenu.org/definitions/114-revenu-net-global.php"

    def function(self, simulation, period):
        ''' Revenu net global (total 20) '''
        period = period.start.offset('first-of', 'year').period('year')
        rbg_individuel = simulation.calculate('rbg_individuel', period)
        csg_deduc = simulation.calculate('csg_deduc', period)
        #print csg_deduc  #TODO : CHeck that, why is it a Foyer size ?
        charges_deduc = simulation.calculate('charges_deduc', period)

        return period, max_(0, rbg_individuel - csg_deduc - charges_deduc)


# In[16]:

class rni_individuel(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Revenu net imposable"
    url = "http://impotsurlerevenu.org/definitions/115-revenu-net-imposable.php"

    def function(self, simulation, period):
        ''' Revenu net imposable ou déficit à reporter'''
        period = period.start.offset('first-of', 'year').period('year')
        rng_individuel = simulation.calculate('rng_individuel', period)
        #abat_spe = simulation.calculate('abat_spe', period)

        return period, rng_individuel #- abat_spe


# In[17]:

class ir_brut_individuel_sans_qf_enfant(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = Individus
    label = u"Impot sur le revenu brut avant non imposabilité et plafonnement du quotient"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('year')
        nbptr_individuel = 1    
        taux_effectif = simulation.calculate('taux_effectif', period)
        rni_individuel = simulation.calculate('rni_individuel', period)
        bareme = simulation.legislation_at(period.start).ir.bareme

        return period, (taux_effectif == 0) * bareme.calc(rni_individuel) + \
                        taux_effectif * rni_individuel


# In[44]:

class ir_brut_individuel_foyer_sans_qf_enfant(Reform_individualisation):
    column = FloatCol(default = 0)
    entity_class = FoyersFiscaux
    label = u""

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('year')
        ir_brut_individuel_holder = simulation.compute('ir_brut_individuel_sans_qf_enfant', period)
        ir_brut_individuel_foyer = self.sum_by_entity(ir_brut_individuel_holder)
        return period, ir_brut_individuel_foyer


        
